In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import time
import requests
import json
from datetime import datetime

In [2]:
def get_btcprice(ticker,max):
    url = ('https://api.coingecko.com/api/v3/coins/')+ticker+('/market_chart?vs_currency=jpy&days=')+max
    r = requests.get(url)
    r2 = json.loads(r.text)
    return r2

In [3]:
# jsonから価格データだけをPandasに変換して抽出する
def get_price(r2):
    s = pd.DataFrame(r2['prices'])
    s.columns = ['date', 'price']
    date = []
    for i in s['date']:
        tsdate = int(i / 1000)
        loc = datetime.utcfromtimestamp(tsdate)
        date.append(loc)
    s.index = date
    del s['date']
    return s

In [8]:
# ビットコインの全期間の価格データを取得する
r2 = get_btcprice('bitcoin', 'max')
btc = get_price(r2)
btc['price'] = btc['price'].astype(int)

In [19]:
btc.index.name = 'ds'
df = btc
df = df.rename(columns={'price':'y'})
df = df.reset_index()
df

,ds,y
0,2013-04-28 00:00:00,13203
1,2013-04-29 00:00:00,14108
2,2013-04-30 00:00:00,13629
3,2013-05-01 00:00:00,11364
4,2013-05-02 00:00:00,10350
...,...,...
3086,2021-10-11 00:00:00,6141845
3087,2021-10-12 00:00:00,6526995
3088,2021-10-13 00:00:00,6396646
3089,2021-10-14 00:00:00,6513561


In [20]:
from prophet import Prophet

In [21]:
m = Prophet()
m.fit(df)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
